In [3]:
!pip install pysabr

  Using cached pysabr-0.4.1-py3-none-any.whl.metadata (321 bytes)
Using cached pysabr-0.4.1-py3-none-any.whl (9.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 9.1 MB/s eta 0:00:00a 0:00:01


In [10]:
!pip install openbb-yfinance

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pysabr as pysabr
from pysabr import Hagan2002LognormalSABR
from pysabr import hagan_2002_lognormal_sabr as sabr
from pysabr.black import lognormal_call

In [3]:
from openbb import obb
from decouple import config

In [8]:
import os
print(os.getcwd())

/Users/saintlaurentjefe/atw


In [11]:
# Get keys and password
api_key = config("API_KEY")

UndefinedValueError: API_KEY not found. Declare it as envvar or define a default value.

In [22]:
symbol = "SPY"
expiration = "2026-01-16"
spy = obb.derivatives.options.chains(symbol,provider='yfinance')

OpenBBError: 
Type -> ValidationError 

Details -> 1 validations errors in chains
Arg provider ->
Input should be 'intrinio' [validation_error_type=literal_error, input_type=str, input_value=yfinance]
    For further information visit https://errors.pydantic.dev/2.7/v/literal_error


In [17]:
spy

,open,high,low,close,volume,split_ratio,dividend,capital_gains
date,,,,,,,,
2023-07-24,453.369995,455.040009,452.299988,454.200012,54023400,0.0,0.0,0.0
2023-07-25,453.920013,456.739990,453.869995,455.440002,55191200,0.0,0.0,0.0
2023-07-26,454.470001,456.989990,453.380005,455.510010,71052900,0.0,0.0,0.0
2023-07-27,459.019989,459.440002,451.549988,452.489990,92194400,0.0,0.0,0.0
2023-07-28,455.880005,457.779999,452.489990,456.920013,80011800,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2024-07-17,558.799988,560.510010,556.609985,556.940002,57119000,0.0,0.0,0.0
2024-07-18,558.510010,559.520020,550.429993,552.659973,56270400,0.0,0.0,0.0
2024-07-19,552.419983,554.080017,547.909973,548.989990,65509100,0.0,0.0,0.0


In [32]:
# Get the last price of the underlying stock.
symbol = "CRM"
provider='intrino'
quote = obb.equity.price.quote(symbol, provider="intrinio")


OpenBBError: 
Type -> OpenBBError

Detail -> Missing credential 'intrinio_api_key'. Check https://intrinio.com/ to get it.

In [30]:
options = obb.derivatives.options.chains(symbol, provider=provider)


OpenBBError: 
Type -> ValidationError 

Details -> 1 validations errors in chains
Arg provider ->
Input should be 'intrinio' [validation_error_type=literal_error, input_type=str, input_value=yfinance]
    For further information visit https://errors.pydantic.dev/2.7/v/literal_error
